In [65]:
import pandas as pd

In [69]:
def concatallfile(company_name,filenumber):
    company = pd.DataFrame()
    for num in range(1,filenumber+1):
        filename = '/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/'+company_name+'/Rdata'+str(num)+'.csv'
        temp_df = pd.read_csv(filename)
        company = pd.concat([company,temp_df],axis = 0)
    
    return company

In [70]:
Foxconn = concatallfile(company_name='鴻海新聞',filenumber=11)
TSMC = concatallfile(company_name='台積電新聞',filenumber=8)
LARGAN = concatallfile(company_name='大立光新聞',filenumber=5)

Foxconn.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/Foxconn.csv',index = None)
TSMC.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/TSMC.csv',index = None)
LARGAN.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/LARGAN.csv',index = None)

讀取檔案

In [412]:
Foxconn = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/Foxconn.csv')
TSMC = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/TSMC.csv')
LARGAN = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/LARGAN.csv')

把日期轉成datetime格式

In [413]:
def datetime_convert(df):
    from datetime import datetime
    df['TIME'] = df['TIME'].apply(lambda x:datetime.strptime(x, "%Y/%m/%d %H:%M"))
    df['TIME'] = df['TIME'].apply(lambda x:x.replace(hour=0, minute=0, second=0, microsecond=0))

    return df

In [414]:
Foxconn = datetime_convert(Foxconn)
TSMC = datetime_convert(TSMC)
LARGAN = datetime_convert(LARGAN)

把不要的欄位去掉

In [415]:
def remove_redundant(df):
    content = ['TITLE','TIME','DESCRIPTION','CONTENT']
    df = df[content]
    
    return df

In [416]:
Foxconn = remove_redundant(Foxconn)
TSMC = remove_redundant(TSMC)
LARGAN = remove_redundant(LARGAN)

將日期作切割，分為"年","月","日","時"


In [417]:
def extract_datetime(df,col):
    df['year'] = df[col].apply(lambda x:x.year)
    df['month'] = df[col].apply(lambda x:x.month)
    df['day'] = df[col].apply(lambda x:x.day)
    df['hour'] = df[col].apply(lambda x:x.hour)
    return df

In [418]:
TSMC = extract_datetime(TSMC,'TIME')
Foxconn = extract_datetime(Foxconn,'TIME')
LARGAN = extract_datetime(LARGAN,'TIME')

## 選出漲跌幅超過2%的日期，提出他前兩日的新聞

In [419]:
#呼叫股票的csv
Foxconn_price = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/2317_鴻海.csv')
TSMC_price = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/2330_台積電.csv')
LARGAN_price = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/3008_大立光.csv')


In [420]:
Foxconn_price

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,107/03/01,"42,073,991","3,698,227,165",87.8,88.3,87.5,87.9,-0.2,"16,467"
1,107/03/02,"28,180,553","2,461,227,316",87.2,87.8,87.0,87.8,-0.1,"13,846"
2,107/03/05,"31,341,174","2,738,562,304",87.7,88.1,87.0,87.0,-0.8,"15,273"
3,107/03/06,"35,392,520","3,124,639,407",88.3,88.8,87.8,88.2,1.2,"14,546"
4,107/03/07,"44,181,640","3,940,688,349",88.6,90.1,88.3,88.5,0.3,"21,638"
...,...,...,...,...,...,...,...,...,...
505,108/10/25,"35,866,261","2,871,800,127",80.7,80.7,79.6,80.2,-0.3,"14,070"
506,108/10/28,"24,589,303","1,976,884,140",80.5,80.8,80.0,80.1,-0.1,"11,179"
507,108/10/29,"43,338,927","3,501,791,877",80.1,81.4,80.1,80.8,0.7,"18,579"
508,108/10/30,"31,199,154","2,503,835,220",80.9,80.9,79.7,80.2,-0.6,"12,486"


處理股價日期較為麻煩，因為他是以民國開頭，因此要先將民國轉換為西元，才可用datetime

In [421]:
#只要把日期的/去掉，轉成int，再加上19110000，就是西元了
def convert_minguo(df,colname):

    df[colname] = df[colname].apply(lambda x:x.split('/'))
    df[colname] = df[colname].apply(lambda x:''.join(x))
    df[colname] = df[colname].apply(lambda x:int(x))
    df[colname] = df[colname].apply(lambda x:x+19110000)
    df[colname] = df[colname].apply(lambda x:datetime.strptime(str(x), "%Y%m%d"))

    return df



In [422]:
TSMC_price = convert_minguo(TSMC_price,'日期')
Foxconn_price = convert_minguo(Foxconn_price,'日期') 
LARGAN_price = convert_minguo(LARGAN_price,'日期') 

由於有些欄位理應是numeric，卻因為含以","被歸類為string，要把這些dtype修正

In [423]:
# 把,拿掉
def strip_comma(df,colist):
    for col in colist:
        if df[col].dtype=='object':
            df[col] = df[col].apply(lambda x:x.replace(',','')).astype(float)
    
    return df
    

In [424]:
Foxconn_price = strip_comma(Foxconn_price,['成交股數','成交金額','開盤價','最高價','最低價','收盤價'])
TSMC_price = strip_comma(TSMC_price,['成交股數','成交金額','開盤價','最高價','最低價','收盤價'])
LARGAN_price = strip_comma(LARGAN_price,['成交股數','成交金額','開盤價','最高價','最低價','收盤價']) 

把每一天都標註是漲還是跌

In [425]:
def find_up_down(df):
    
    status = []
    for i in range(df.shape[0]):
        if i==0:
            status.append(0)
        
        elif df.iloc[i,6] > 1.02*df.iloc[i-1,4]:#如果當天的收盤價 > 前一日最高價，表示當日受前一日影響，漲價
            status.append(1)
        
        elif df.iloc[i,4]*1.02 < df.iloc[i-1,6]:  #如果當天的收盤價 < 前一日最高價，表示當日受前一日影響，跌價
            status.append(-1)
        else:
            status.append(0)
    df['status'] = status

    return df

In [426]:
Foxconn_price = find_up_down(Foxconn_price)
TSMC_price = find_up_down(TSMC_price)
LARGAN_price = find_up_down(LARGAN_price)


挑出漲跌的那些日子

In [444]:
'''
前面已經提到，我們透過一些標準將每一天股市分成漲、跌、不漲不跌，現在我們希望把這些日期的標籤貼到新聞上
賦予每篇新聞一個屬性(看漲新聞、看跌新聞、不漲不跌新聞)
(Note:由於每天的開盤我們只會得到開盤日以前的新聞，
因此假設11/10是上漲日，那麼我們會對11/9和11/8的文章貼上status=1標籤)
'''

def find_pos_neg_day(price_df,news_df):

    from datetime import timedelta
    
    upday1 = list(price_df[price_df['status']==1]['日期'] - timedelta(days=1)) #當天股價上漲，表示前幾天的新聞一定有掛
    upday2 = list(price_df[price_df['status']==1]['日期'] - timedelta(days=2))
    upday = upday1 + upday2

    downday1 = list(price_df[price_df['status']==-1]['日期'] - timedelta(days=1)) #當天股價下跌，表示前幾天的新聞一定有掛
    downday2 = list(price_df[price_df['status']==-1]['日期'] - timedelta(days=2))
    downday = downday1 + downday2 
    news_df['status'] = news_df['TIME'].apply(lambda x:1 if x in upday else(-1 if x in downday else 0)) #為新聞貼上標籤

    return news_df

In [450]:
LARGAN = find_pos_neg_day(LARGAN_price, LARGAN) 
Foxconn = find_pos_neg_day(Foxconn_price, Foxconn)
TSMC = find_pos_neg_day(TSMC_price, TSMC)


In [453]:
LARGAN.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/大立光正負新聞.csv',index = None)
Foxconn.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/鴻海正負新聞.csv',index = None)
TSMC.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/台積電正負新聞.csv',index = None)

剩下的我們到下一個jieba.py在來提吧!